### Import

---

In [6]:
import numpy as np
from note_include.elem.Ring  import Ring
from note_include.elem.RLWE  import RLWE
from note_include.elem.RLWEp import RLWEp
np.set_printoptions(linewidth=np.inf) # print np array in only one line


---

# $\textsf{RGSW}$

* <a href="https://eprint.iacr.org/2014/816"> FHEW: Bootstrapping Homomorphic Encryption in less than a second </a>

* <a href="https://eprint.iacr.org/2020/086"> Bootstrapping in FHEW-like Cryptosystems </a>

---

앞서 배웠던 RLWE' 은 gadget decomposition 을 활용하여 다항식의 계수의 값을 크게 줄여 노이즈를 관리하는 측면에서 일반적인  $\text{RLWE} \circ \mathcal{R}_Q$ 보다 효율적인 모습을 보였다.

그러나 RLWE' 은 평문 다항식 곱셈을 효율적으로 수행하기 위해 고안된 것이며 RLWE' 은 별도로 동형 곱셈을 지원하지는 않는다.

우리의 최종 목표는 동형 곱셈을 효율적으로 수행하도록 만드는 것이며, 이를 위해 이번에는 RLWE' 에 기반하고 있는 RGSW 를 새로 정의하여 동형 곱셈을 효율적으로 만들 것이다.

암호화 부터 정의해보자.

>$\textbf{Definition : RGSW encryption}$
>
>어떠한 메세지 다항식 $\boldsymbol{m} \in \mathcal{R}_Q$ 와 비밀키 다항식 $\boldsymbol{s} \in \mathcal{R}_Q$ 가 있다고 하자.
>
>이 경우 RGSW 암호화는 다음과 같이 정의할 수 있다:
>
>\begin{align*}
>    \textsf{RGSW.Enc}(\boldsymbol{m}) = 
>    \left( 
>        \textsf{RLWE'.Enc}_B(-\boldsymbol{s} \cdot \boldsymbol{m}), 
>        \textsf{RLWE'.Enc}_B(\boldsymbol{m}) \in \mathcal{R}_Q^{2 \times 2 \times d}
>     \right)
>\end{align*}
>
>여기서 $B$ 는 gadget decomposition 에 사용되는 base value 이며, $d$ 는 $d = \lceil \log_BQ \rceil$ 이다.

코드로 확인해보자.

In [7]:
def rgsw_encrypt(CC:RLWEp, m:Ring, s:Ring) -> tuple[ list[tuple[Ring, Ring]], list[tuple[Ring, Ring]] ]:
    e0 = CC.encrypt(-1 * s * m, s)
    e1 = CC.encrypt(m, s)

    return [e0, e1]

In [8]:
N   = 16
Q   = 128
B   = 2
d   = int(np.ceil(np.log(Q) / np.log(B)))
std = 0

RLWE_CC  = RLWE(N, Q, s_std='Gaussian', e_std=3.2) #
RLWEp_CC = RLWEp(N, Q, 'Gaussian', 3.2, B, d)

s, _ = RLWE_CC.keygen() # secret key, public key
m    = Ring(N, Q, [i for i in range(N)])

rgsw_ctxt = rgsw_encrypt(RLWEp_CC, m, s)


---

왜 저런식으로 정의를 해야하는지 의문일 수 있다.

이는 동형 곱셈을 수행하기 위해 정의한 것이므로 동형 곱셈의 정의를 보고 나서야 이해할 수 있을 것이다.

RLWE 와 RGSW 간의 곱셈 $(\odot)$ 을 다음과 같이 정의해보자.

>$\textbf{Definition : RLWE } \odot \textbf{ RGSW (Homomorphic multiplication)}$
>
>RLWE 암호문 $(\boldsymbol{a}, \boldsymbol{b}) \in \mathcal{R}_Q^2$ 과 
>RGSW 암호문 $(\boldsymbol{E}_0, \boldsymbol{E}_1) \in \mathcal{R}_Q^{2 \times 2 \times d}$ 이 있다고 하자.
>
>이 때 두 암호문 간의 곱셈$(\odot)$은 다음과 같이 정의할 수 있다.
>
>\begin{align*}
>    & (\odot) : \text{RLWE} \odot \text{RGSW} \rightarrow \text{RLWE} \\
>    & \text{RLWE} \odot \text{RGSW} = \boldsymbol{a} \ast \boldsymbol{E}_0 + \boldsymbol{b} \ast \boldsymbol{E}_1
>\end{align*}
>
>여기서 ($\ast$) 는 RLWE 와 RLWE' 간의 곱셈 연산이다.

위의 정의가 타당한지 확인해보자.

>$\textbf{Correctness : RLWE } \odot \textbf{ RGSW }$
>
>RLWE 암호문은 $\boldsymbol{m}_0$, RGSW 암호문은 $\boldsymbol{m}_1$ 을 각각 $\boldsymbol{s}$ 으로 암호화 하고 있다고 하자.
>
>이 경우 다음 수식이 성립함을 알 수 있다.
>
>\begin{align*}
>    \text{RLWE} \odot \text{RGSW} &= 
>        \boldsymbol{a} \ast \text{RLWE}'(-\boldsymbol{s}\boldsymbol{m}_1) + 
>        \boldsymbol{b} \ast \text{RLWE}'(\boldsymbol{m}_1) \\ &=
>        \text{RLWE}(- \boldsymbol{a} \boldsymbol{s} \boldsymbol{m}_1 ) +
>        \text{RLWE}(\boldsymbol{b} \boldsymbol{m}_1 ) \\ &=
>        \text{RLWE}( \boldsymbol{b} \boldsymbol{m}_1 - \boldsymbol{a} \boldsymbol{s} \boldsymbol{m}_1 ) \\ &=
>        \text{RLWE}( ( \boldsymbol{b} - \boldsymbol{a} \boldsymbol{s} ) \cdot \boldsymbol{m}_1 ) \\ &=
>        \text{RLWE}( (m_0 + e_0) \cdot m_1 ) \\&= 
>        \text{RLWE}( m_0 \cdot m_1 + e_0 \cdot m_1 ) \\&
>        \approx \text{RLWE}( m_0 \cdot m_1 ).
>
>
>\end{align*}

즉, 암호문에 있는 다항식을 그대로 곱해버리는 것이다.

암호문은 무작위 난수처럼 보이기에 계수가 $\mathbb{Z}_Q$ 상에서 무작위로 나타나는 것과 동일하여 일반적인 곱셈을 수행하게 될 경우 노이즈가 매우 커지지만,

앞서 정의한 RLWE' 을 활용하여 이 과정에서 발생하는 노이즈를 최소화 한다.

이를 통해서 동형 곱셈을 정의할 수 있게 된다.

코드를 통해서 확인해보자.

In [9]:
def rgsw_mult_rlwe(CC_RLWEp:RLWEp, CC_RLWE:RLWE, rlwe_ctxt:tuple[Ring, Ring], rgsw_ctxt:tuple[list[tuple[Ring, Ring]],list[tuple[Ring, Ring]] ]) -> tuple[Ring, Ring]:
    a,   b = rlwe_ctxt
    e0, e1 = rgsw_ctxt

    tmp1 = CC_RLWEp.mult_poly(e0, a) # -> Return RLWEctxt
    tmp2 = CC_RLWEp.mult_poly(e1, b) # -> Return RLWEctxt

    res  = CC_RLWE.add_ctxt_ctxt(tmp1, tmp2)
    return res

# 유틸리티
def crange(coeffs, q):
    coeffs = np.where((coeffs >= 0) & (coeffs <= q//2), coeffs, coeffs - q)
    return coeffs

In [10]:
N   = 16
Q   = 7681
B   = 4
d   = int(np.ceil(np.log(Q) / np.log(B)))
std = 3.2

RLWE_CC  = RLWE(N, Q, s_std='Gaussian', e_std=3.2) #
RLWEp_CC = RLWEp(N, Q, 'Gaussian', 3.2, B, d)

s, _ = RLWE_CC.keygen() # secret key, public key
m0   = Ring(N, Q, [300 for i in range(N)])
m1   = Ring(N, Q, [3  for i in range(N)])

rlwe_ctxt = RLWE_CC.encrypt(m0, s)
rgsw_ctxt = rgsw_encrypt(RLWEp_CC, m1, s)

ctxt_mult  = rgsw_mult_rlwe(RLWEp_CC, RLWE_CC, rlwe_ctxt, rgsw_ctxt)
ptxt_mult  = RLWE_CC.decrypt(ctxt_mult, s)
rgsw_noise = crange(np.array((m0 * m1 - ptxt_mult).coeffs), Q)

print("Ideal Result : ", m0 * m1)
print("Plaintext    : ", ptxt_mult)
print("RGSW Noise   : ", rgsw_noise)

Ideal Result :  (2762 + 4562x + 6362x^2 + 481x^3 + 2281x^4 + 4081x^5 + 5881x^6 + 0x^7 + 1800x^8 + 3600x^9 + 5400x^10 + 7200x^11 + 1319x^12 + 3119x^13 + 4919x^14 + 6719x^15 | n=16, q=7681)
Plaintext    :  (2900 + 4598x + 6384x^2 + 449x^3 + 2289x^4 + 4094x^5 + 5831x^6 + 7673x^7 + 1713x^8 + 3583x^9 + 5310x^10 + 7167x^11 + 1233x^12 + 2990x^13 + 4839x^14 + 6668x^15 | n=16, q=7681)
RGSW Noise   :  [-138  -36  -22   32   -8  -13   50    8   87   17   90   33   86  129   80   51]



---

RLWE 곱셈과 노이즈를 비교해보자.

In [11]:
# 정규분포(Gaussin distribution)를 따르는 난수 생성기
def discrete_gaussian(n, q, mean=0., std=3.2):
    coeffs = np.round(std * np.random.randn(n)) % q
    return np.array(coeffs, dtype = int)

# 균등분포(Uniform distribution)를 따르는 난수 생성기
def discrete_uniform(n, q, min=0., max=None):
    if max is None:
        max = q
    coeffs = np.random.randint(min, max, size=n)
    return np.array(coeffs, dtype = int)

# RLWE 암호화
def RLWE_Enc(m:Ring, s:Ring, std=3.2) -> tuple[Ring, Ring]:
    N = m.n
    Q = m.q
    a = Ring(N, Q, discrete_uniform(N, Q))       # Random Number
    e = Ring(N, Q, discrete_gaussian(N, Q, std=std)) # Noise

    b = a * s + m + e
    return (a, b)

def RLWE_Mul_Ciphertext(ct1:tuple[Ring, Ring], ct2:tuple[Ring, Ring]) -> tuple[Ring, Ring, Ring]:
    a1, b1 = ct1
    a2, b2 = ct2

    d0 = a1 * b2 + a2 * b1
    d1 = a1 * a2
    d2 = b1 * b2

    return [d0, d1, d2]

def RLWE_Dec_Mul(ct:tuple[Ring, Ring, Ring], s:Ring) -> Ring:
    d0, d1, d2 = ct
    msg = d2 - (d0*s) + (d1*s*s)

    return msg

In [12]:
N   = 16
Q   = 7681
B   = 4
d   = int(np.ceil(np.log(Q) / np.log(B)))
std = 3.2

RLWE_CC  = RLWE(N, Q, s_std='Gaussian', e_std=3.2) #
RLWEp_CC = RLWEp(N, Q, 'Gaussian', 3.2, B, d)

s, _ = RLWE_CC.keygen() # secret key, public key
m0   = Ring(N, Q, [300 for i in range(N)])
m1   = Ring(N, Q, [3  for i in range(N)])

### RLWE x RGSW
rlwe_ctxt = RLWE_CC.encrypt(m0, s)
rgsw_ctxt = rgsw_encrypt(RLWEp_CC, m1, s)

ctxt_mult  = rgsw_mult_rlwe(RLWEp_CC, RLWE_CC, rlwe_ctxt, rgsw_ctxt)
ptxt_mult  = RLWE_CC.decrypt(ctxt_mult, s)
rgsw_noise = crange(np.array(((m0 * m1) - ptxt_mult).coeffs), Q)

### RLWE x RLWE
ct1     = RLWE_Enc(m0, s)
ct2     = RLWE_Enc(m1, s)

ct_mul     = RLWE_Mul_Ciphertext(ct1, ct2) 
pt_mul     = RLWE_Dec_Mul(ct_mul, s)
rlwe_noise = crange(np.array(((m0 * m1) - pt_mul).coeffs), Q)

print("RGSW Noise   : ", rgsw_noise)
print("RLWE Noise   : ", rlwe_noise)

RGSW Noise   :  [ 98 208 174 129 136  69 123 135 102  89  91 199  55 -60  -9   4]
RLWE Noise   :  [-2343    17 -2396 -1800 -1263 -2972    50    76   -50   662  2353  2372  -119    32  1100   -87]



---

위를 확인해보면 확실히 RLWE 곱셈에 비해 노이즈가 잘 잡히는 모습을 확인할 수 있다.

참고할 것이 있다면 RGSW 또한 너무 큰 값을 암호화 하여 동형연산을 수행할 경우 노이즈가 잘 안잡히는 모습을 보인다. 
(m1의 계수를 [300 for _ in range(N)] 으로 바꾸어 실행해보면 알 수 있다.)

그럼에도 FHEW 에서 RGSW 곱셈을 사용하는 이유는 FHEW 의 핵심인 blind rotation 이라는 기술은 큰 값을 곱하지 않기 때문에 그렇다.
(주로 monomial 을 곱하게 된다.)

---

## Summary

이번에는 RLWE' 을 활용한 RGSW 에 대해서 알아보았으며 어떻게 동형 곱셈을 효율적으로 수행하는지에 대해서 알아보았다.

한계점은 명확한 것이 RGSW 로 암호화 하는 메세지가 클 경우 노이즈가 크게 잡히지 않는 모습을 보이고 있다.

그러나 FHEW 에서는 큰 값을 곱하지 않기 때문에 RGSW 곱셈을 활용하고 있으며,

이전에 RLWE 에서 설명했던 곱셈과는 달리 차원이 늘어나지 않는다는 특징을 갖고 있다.


---

## Code implementation

`note_include/elem/RGSW.py` 를 확인하면 위에서 정의한 연산들이 있는 RGSW 구현을 확인할 수 있다.